In [1]:
import torch

In [3]:
#tensoを作成するときに、requires_gradを指定すると、勾配計算をメモリに保存しといてくれる。デフォでは、メモリ削減のため保存しない。
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [9]:
#requires_gradがTrueのtensorを計算すると、引き継がれる。
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [10]:
#演算の結果としてyが生成され、grad_fnをもつ。
print(y.grad_fn)

In [11]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [12]:
#requires_grad_で、flagを書き換えられる。デフォルトでは、False。
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


In [15]:
#backward()で誤差逆伝播を計算できる。一度backwardを実行すると、順伝播の計算結果がメモリから解放される。ので、second timeはエラーがでる。
out.backward()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [14]:
out.backward(torch.tensor(1.))

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [20]:
#d(out)/dxの勾配計算。
#x.gradはbackwardに送られたoutをxで偏微分することを意味する。
print(x.grad)

None


In [18]:
x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([  -22.1310,  -148.3490, -1159.2367], grad_fn=<MulBackward0>)


In [21]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
#dot(y, v.T)を勾配計算。
y.backward(v)
#yをxで偏微分したもの。x.shapeが3なので、x.grad.shapeも3。
print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [22]:
#with torch.no_gradであれば、requires_gradをFalseにしなくてもtracking historyをストップできるよ。
print(x.requires_grad)
print((x ** 2).requires_grad)
with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


In [23]:
#detachすると、tracking historyはコピーせずに、オブジェクトのみをコピーできるよ。
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
